In [3]:
import os
import sys

import openai
from dotenv import load_dotenv

import psycopg


from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate,PromptTemplate
from langchain.chains import ConversationChain, LLMChain
from langchain.chains.conversation.memory import (ConversationBufferMemory, 
                                                  ConversationSummaryMemory, 
                                                  ConversationBufferWindowMemory,
                                                  ConversationEntityMemory)
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.callbacks import get_openai_callback

from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent


OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')



In [4]:
llms = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=OPENAI_API_KEY)

In [5]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
    return result

### Testing for duplicated memories

#### ConversationBufferMemory

In [6]:
conversation_buf = ConversationChain(
    llm=llms,
    memory=ConversationBufferMemory()
)

In [7]:
conversation_buf("I am going outside.")

{'input': 'I am going outside.',
 'history': '',
 'response': 'That sounds like a great idea! What are you planning to do outside?'}

In [8]:
count_tokens(
    conversation_buf, 
    "I have returned home and am going to take a bath"
)

Spent a total of 123 tokens


'That sounds relaxing! Enjoy your bath. Do you have any specific bath products or scents that you like to use?'

In [9]:
count_tokens(
    conversation_buf, 
    "I am going outside again"
)

Spent a total of 148 tokens


'Oh, are you going out for a walk or to run errands?'

#### Conversational Summary Memory

In [10]:
conversation_sum = ConversationChain(
    llm=llms, 
    memory=ConversationSummaryMemory(llm=llms)
)

In [11]:
count_tokens(
    conversation_sum, 
    'I am going outside.'
)

Spent a total of 261 tokens


'That sounds like a great idea! What are you planning to do outside?'

In [12]:
count_tokens(
    conversation_sum, 
    "I have returned home and am going to take a bath"
)

Spent a total of 398 tokens


'That sounds like a relaxing plan! Taking a bath can be a great way to unwind after being outside. Do you have any specific bath products or rituals that you enjoy?'

In [13]:
count_tokens(
    conversation_sum, 
    "I am going outside again"
)

Spent a total of 385 tokens


"That's great! What are your plans for outside?"

#### ConversationBufferWindowMemory 

Keeping the few conversations, Uses the the latest conversation and drops the previous conversation.

In [14]:
conversation_bufw = ConversationChain(
    llm=llms, 
    memory=ConversationBufferWindowMemory(k=2)
)

In [15]:
count_tokens(
    conversation_bufw, 
    'I am going outside.'
)

Spent a total of 83 tokens


'That sounds like a great idea! What are you planning to do outside?'

In [16]:
count_tokens(
    conversation_bufw, 
    "I have returned home and am going to take a bath"
)

Spent a total of 132 tokens


'That sounds relaxing! Taking a bath can be a great way to unwind after being outside. Do you have any specific bath products or scents that you enjoy using?'

In [17]:
count_tokens(
    conversation_bufw, 
    "I am going outside again"
)

Spent a total of 161 tokens


'Oh, are you going back outside so soon? What are you planning to do this time?'

#### ConversationEntityMemory

In [18]:
memory = ConversationEntityMemory(llm=llms)
conversation_entity= ConversationChain(
    llm=llms, 
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=memory
)

In [19]:
count_tokens(
    conversation_entity, 
    'I am going outside.'
)

Spent a total of 777 tokens


"That sounds like a great idea! Going outside can be a refreshing and rejuvenating experience. Whether you're going for a walk, spending time in nature, or engaging in outdoor activities, being outside can have numerous benefits for both physical and mental well-being. It allows you to get some fresh air, soak up some vitamin D from the sun, and enjoy the beauty of the natural world. Just make sure to dress appropriately for the weather and stay safe during your outdoor adventures. Have a wonderful time outside!"

In [20]:
count_tokens(
    conversation_entity, 
     "I have returned home and am going to take a bath"
)

Spent a total of 1020 tokens


"Welcome back! Taking a bath after being outside can be a great way to relax and unwind. It can help soothe tired muscles, relieve stress, and provide a sense of calm and tranquility. You can enhance your bath experience by adding some bath salts, essential oils, or bubble bath to create a luxurious and aromatic atmosphere. Don't forget to set the water temperature to your liking and make sure to have a towel and any other bath essentials within reach. Enjoy your bath and let me know if there's anything else I can assist you with!"

In [21]:
count_tokens(
    conversation_entity, 
     "I am going outside again"
)

Spent a total of 1254 tokens


"That's great to hear! Going outside again can provide another opportunity to enjoy the benefits of being in nature and getting some fresh air. Whether you're going for a walk, engaging in outdoor activities, or simply spending time in your backyard, being outside can help improve your mood, boost your energy levels, and provide a change of scenery. Just remember to dress appropriately for the weather and stay safe during your outdoor adventures. Have a wonderful time outside! Let me know if there's anything else I can help you with."

In [22]:
memory.chat_memory.messages

[HumanMessage(content='I am going outside.'),
 AIMessage(content="That sounds like a great idea! Going outside can be a refreshing and rejuvenating experience. Whether you're going for a walk, spending time in nature, or engaging in outdoor activities, being outside can have numerous benefits for both physical and mental well-being. It allows you to get some fresh air, soak up some vitamin D from the sun, and enjoy the beauty of the natural world. Just make sure to dress appropriately for the weather and stay safe during your outdoor adventures. Have a wonderful time outside!"),
 HumanMessage(content='I have returned home and am going to take a bath'),
 AIMessage(content="Welcome back! Taking a bath after being outside can be a great way to relax and unwind. It can help soothe tired muscles, relieve stress, and provide a sense of calm and tranquility. You can enhance your bath experience by adding some bath salts, essential oils, or bubble bath to create a luxurious and aromatic atmosp

In [23]:
count_tokens(
    conversation_buf, 
    "Imagine: You are AI Resarcher. You need to Optimize the model for the best perfomance. You think about the best way to do it. How do you do it?"
)

Spent a total of 432 tokens


"As an AI researcher, optimizing a model for the best performance involves several steps. First, I would analyze the current model's performance and identify any areas that need improvement. This could include issues such as slow training or inference times, high memory usage, or low accuracy.\n\nNext, I would explore different optimization techniques and algorithms that are suitable for the specific model and task at hand. This could involve techniques such as gradient descent optimization, weight pruning, quantization, or model compression.\n\nI would also consider the hardware and software infrastructure available for training and deploying the model. This could include utilizing specialized hardware accelerators like GPUs or TPUs, or leveraging distributed computing frameworks like TensorFlow or PyTorch.\n\nAdditionally, I would experiment with different hyperparameters, such as learning rate, batch size, or regularization techniques, to find the optimal settings for the model.\n\n

In [24]:
count_tokens(
    conversation_buf, 
    "Why do you think that is the best way to do it?"
)

Spent a total of 696 tokens


"As an AI researcher, I believe that the approach I described is a good way to optimize a model for the best performance because it involves a systematic and iterative process. By analyzing the current model's performance and identifying areas for improvement, we can target specific issues and make targeted optimizations.\n\nExploring different optimization techniques and algorithms allows us to leverage the latest advancements in the field and apply them to our specific model and task. This helps us find the most effective methods for improving performance.\n\nConsidering the hardware and software infrastructure is important because it allows us to leverage the available resources efficiently. Specialized hardware accelerators and distributed computing frameworks can significantly speed up training and inference times, leading to better performance.\n\nExperimenting with different hyperparameters helps us fine-tune the model and find the optimal settings. This allows us to strike a ba

In [25]:
count_tokens(
    conversation_sum, 
    "Imagine: You are AI Resarcher. You need to Optimize the model for the best perfomance. You think about the best way to do it. How do you do it?"
)

Spent a total of 1058 tokens


"As an AI researcher, optimizing a model for the best performance involves several steps. First, I would analyze the current model's performance and identify any areas that need improvement. This could include issues such as slow inference times, high memory usage, or low accuracy.\n\nNext, I would explore different optimization techniques such as pruning, quantization, or knowledge distillation. Pruning involves removing unnecessary connections or neurons from the model to reduce its size and improve efficiency. Quantization, on the other hand, involves reducing the precision of the model's weights and activations, which can lead to faster computations. Knowledge distillation is a technique where a smaller, more efficient model is trained to mimic the behavior of a larger, more accurate model.\n\nAdditionally, I would experiment with different hyperparameters, such as learning rate, batch size, and regularization techniques, to find the optimal configuration for the model. This could 

In [26]:
count_tokens(
    conversation_sum, 
    "Why do you think that is the best way to do it?"
)

Spent a total of 1330 tokens


"As an AI researcher, I believe that optimizing a model for the best performance involves a combination of careful analysis, experimentation, and fine-tuning because it allows us to systematically improve the model's performance. By analyzing the model's current performance, we can identify specific areas that need improvement. This analysis helps us understand the strengths and weaknesses of the model, which is crucial for making informed decisions on how to optimize it.\n\nExperimentation is also essential because it allows us to explore different techniques and approaches to improve the model's performance. Techniques such as pruning, quantization, and knowledge distillation can be applied to reduce the model's size, improve its efficiency, and enhance its generalization capabilities. By experimenting with these techniques, we can determine which ones work best for a particular model and problem.\n\nFine-tuning is another crucial aspect of optimizing a model. This involves adjusting

In [27]:
count_tokens(
    conversation_entity, 
    "Imagine: You are AI Resarcher. You need to Optimize the model for the best perfomance. You think about the best way to do it. How do you do it?"
)

Spent a total of 1922 tokens


"As an AI researcher, optimizing the model for the best performance would involve several steps. Here's a general approach that can be taken:\n\n1. Data preprocessing: Start by analyzing and preprocessing the training data. This may involve cleaning the data, removing noise, handling missing values, and normalizing the data to ensure consistency and improve model performance.\n\n2. Model architecture: Consider the architecture of the model itself. This includes selecting the appropriate neural network architecture, such as recurrent neural networks (RNNs), transformers, or convolutional neural networks (CNNs), depending on the nature of the task. Experiment with different architectures and hyperparameters to find the best combination for your specific use case.\n\n3. Training process: Optimize the training process by selecting an appropriate loss function, regularization techniques, and optimization algorithms. Experiment with different learning rates, batch sizes, and training epochs 

In [28]:
count_tokens(
    conversation_entity, 
    "Why do you think that is the best way to do it?"
)

Spent a total of 2317 tokens


"As an AI language model, I don't have personal opinions or beliefs. The approach I described is a general framework that AI researchers often follow when optimizing models for performance. It combines various techniques and strategies that have been proven effective in the field of machine learning. However, it's important to note that the best way to optimize a model can vary depending on the specific task, dataset, and constraints. Researchers often experiment with different approaches and iterate on the optimization process to find the best solution for their particular use case."